## Top Cited Papers by an Author

In [8]:
import json
author_citations = json.load(open("author_citations.json", "r"))

In [23]:
#Required input author id
ai2_id = input("Please enter the AI2 id of the author for further processing:")

Please enter the AI2 id of the author for further processing:48323507


In [24]:
import json
import pandas as pd

if ai2_id in author_citations:
    top_cited_papers_json = author_citations[ai2_id]["papers"]
    if top_cited_papers_json:
        top_cited_df = pd.DataFrame(top_cited_papers_json)
else:
    print(f"Sorry! we don't seem to have info about this author: {ai2_id}")

## Papers metadata

In [25]:
required_fields = ["title", "abstract", "url", "tldr"]
s2api_url = f"https://api.semanticscholar.org/graph/v1/paper/{{}}?fields={','.join(required_fields)}"

In [26]:
reqd_paper_ids = list(top_cited_df["paper_sha"].unique())

In [27]:
from tqdm import tqdm
import requests
s2api_responses = []
failed_ids = []
for pid in tqdm(reqd_paper_ids):
    res = requests.get(s2api_url.format(pid))
    try:
        if res.status_code == 200:
            res_json = json.loads(res.content)
            res_json["tldr"] = res_json["tldr"]["text"]
            s2api_responses.append(res_json)
        else:
            print(res.content)
            failed_ids.append(pid)
    except:
        print(f"Error received for paper id: {pid}")
        failed_ids.append(pid)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:29<00:00,  1.17s/it]


In [28]:
if failed_ids:
    print(failed_ids)
    print("The request for above paper ids failed, you might want to try again...")
paper_text_df = pd.DataFrame(s2api_responses)
    

In [29]:
def make_clickable(url):
    return '<a href="{}">{}</a>'.format(url,url)

In [30]:
final_paper_df = top_cited_df.merge(paper_text_df, left_on="paper_sha", right_on="paperId", suffixes=[None, "_y"]).drop(["title_y", "paperId", "abstract"], axis=1)

In [31]:
final_paper_df["url"] = final_paper_df["url"].apply(lambda x: f'<a href="{x}">{x}</a>')

### Final dataframe

In [32]:
from pretty_html_table import build_table
html_table_blue_light = build_table(final_paper_df, 'blue_light', escape=False)

In [33]:
author_citations[ai2_id]["name"]

'Peter Clark'

In [34]:
from IPython.display import display, HTML

display(HTML(html_table_blue_light))

paper_sha,corpus_paper_id,title,n_citations,g_citations,year,self_cited,url,tldr
3cfb319689f06bf04c2e28399361f414ca32c4b3,204838007,Exploring the Limits of Transfer Learning with a Unified Text-to-Text Transformer,20,6010,2020,False,https://www.semanticscholar.org/paper/3cfb319689f06bf04c2e28399361f414ca32c4b3,"This systematic study compares pre-training objectives, architectures, unlabeled datasets, transfer approaches, and other factors on dozens of language understanding tasks and achieves state-of-the-art results on many benchmarks covering summarization, question answering, text classification, and more."
05dd7254b632376973f3a1b4d39485da17814df5,11816014,"SQuAD: 100,000+ Questions for Machine Comprehension of Text",19,5120,2016,False,https://www.semanticscholar.org/paper/05dd7254b632376973f3a1b4d39485da17814df5,"A strong logistic regression model is built, which achieves an F1 score of 51.0%, a significant improvement over a simple baseline (20%)."
df2b0e26d0599ce3e70df8a9da02e51594e0e992,52967399,BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding,18,46386,2019,False,https://www.semanticscholar.org/paper/df2b0e26d0599ce3e70df8a9da02e51594e0e992,"A new language representation model, BERT, designed to pre-train deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers, which can be fine-tuned with just one additional output layer to create state-of-the-art models for a wide range of tasks."
d87ceda3042f781c341ac17109d1e94a717f5f60,5958691,WordNet : an electronic lexical database,17,13882,2000,False,https://www.semanticscholar.org/paper/d87ceda3042f781c341ac17109d1e94a717f5f60,"The lexical database: nouns in WordNet, Katherine J. Miller a semantic network of English verbs, and applications of WordNet: building semantic concordances are presented."
478b4a5123bd5fda98bb35e6317d7f3555fec97d,1255845,"Combining Retrieval, Statistics, and Inference to Answer Elementary Science Questions",16,127,2016,True,https://www.semanticscholar.org/paper/478b4a5123bd5fda98bb35e6317d7f3555fec97d,"This paper describes an alternative approach that operates at three levels of representation and reasoning: information retrieval, corpus statistics, and simple inference over a semi-automatically constructed knowledge base, to achieve substantially improved results."
88bb0a28bb58d847183ec505dda89b63771bb495,3922816,"Think you have Solved Question Answering? Try ARC, the AI2 Reasoning Challenge",14,344,2018,True,https://www.semanticscholar.org/paper/88bb0a28bb58d847183ec505dda89b63771bb495,"A new question set, text corpus, and baselines assembled to encourage AI research in advanced question answering constitute the AI2 Reasoning Challenge (ARC), which requires far more powerful knowledge and reasoning than previous challenges such as SQuAD or SNLI."
b4317b8a4490c84301907a61f5b8ebb26ab8828d,12363172,Discovery of inference rules for question-answering,14,637,2001,False,https://www.semanticscholar.org/paper/b4317b8a4490c84301907a61f5b8ebb26ab8828d,"This paper presents an unsupervised algorithm for discovering inference rules from text based on an extended version of Harris’ Distributional Hypothesis, which states that words that occurred in the same contexts tend to be similar."
498bb0efad6ec15dd09d941fb309aa18d6df9f5f,207169186,Open Information Extraction from the Web,13,2376,2008,False,https://www.semanticscholar.org/paper/498bb0efad6ec15dd09d941fb309aa18d6df9f5f,"Open IE (OIE), a new extraction paradigm where the system makes a single data-driven pass over its corpus and extracts a large set of relational tuples without requiring any human input, is introduced."
00ee2e3f19be3be3a0303ac08ed5fc60e0d38c69,5255813,Building Concept Representations from Reusable Components,13,84,1997,True,https://www.semanticscholar.org/paper/00ee2e3f19be3be3a0303ac08ed5fc60e0d38c69,"This work extends the normal inheritance approach used in frame-based systems, and imports ideas 